As you can see below, there are (at least) 3 NB models:

In [4]:
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB

import pandas as pd
import numpy as np

import string
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords 

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\gravi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


It is not entirely clear what the difference is between them, but I can assume that MultinomialNB will not fit my case.

### Nvm here's the description:

We use BernoulliNB here, as we have all 1/0 data - GaussianNB stores mean and standard deviation (sd) of each value for a continuous variables, and Mulitnomial allows multiple categorical values.

In [ ]:
# same shit as usual, instantiate, fit
# Instatiate our model
nbmodel = BernoulliNB()
# Fit our model
nbmodel.fit(x,y)

It seems that once it is fit, all you need is to test(predict) the new(test) data.

In [ ]:
print(nbmodel.predict_proba(new_data) )#get probabilistic prediction
print(nbmodel.predict(new_data) )#get hard prediction

Plus you can get a score, whatever that means. In the example output, the score was 1.0, whereas the probabilites were .42 and .57 so.. kinda confusing. score 1.0 kinda smells like values between 0 and 1. Remains to be seen.

In [ ]:
print("score",nbmodel.score(x,y)) # <<-- score

The way this makes predictions makes me feel like it definitely fits my case:

In [ ]:
X_predictions['proba_is_not_bird'] = np.round(soft_prediction[:, 0], 3)
X_predictions['proba_is_bird'] = np.round(soft_prediction[:, 1], 3) # I guess columns 0 and 1 in this case are prob class 0,
# prob class 1
X_predictions
# and again, predict_proba for soft predictions and predict for hard.

#### On the flip side, although naive Bayes is known as a decent classifier, it is known to be a bad estimator, so the probability outputs from predict_proba are not to be taken too seriously.

lol

#### In the case of text classification, word occurrence vectors (rather than word count vectors) may be used to train and use this classifier. BernoulliNB might perform better on some datasets, especially those with shorter documents. It is advisable to evaluate both models, if time permits.

## Let us begin:

In [6]:
reddit = pd.read_csv('reddit_final.csv')

In [7]:
X = reddit['comment']
y = reddit['label']

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [12]:
stemmer = nltk.stem.PorterStemmer()
stop_words = stopwords.words('english')

def tokenizer(sentence):
    
    num_range = range(0,31)
    
    for num in num_range:
        sentence = sentence.replace(str(num), '')
    
    # remove punctuation and set to lower case
    for punctuation_mark in string.punctuation:
        sentence = sentence.replace(punctuation_mark,'').lower()
        
    
    # split sentence into words
    words_list = sentence.split(' ')
    tokens = []
    
    
    # remove stopwords and any tokens that are just empty strings
    for word in words_list:
        if (not word in stop_words) and (word!=''):
            # Stem words
            stemmed_word = stemmer.stem(word)
            tokens.append(stemmed_word)

    return tokens

In [13]:
tfidf = TfidfVectorizer(min_df=25,
                        tokenizer=tokenizer, max_features=1500, dtype=np.float32)
tfidf.fit(X_train)

X_train_tfidf = tfidf.transform(X_train)
#X_val_tfidf = tfidf.transform(X_val)
X_test_tfidf = tfidf.transform(X_test)

In [14]:
train_tfidf = pd.DataFrame(columns=tfidf.get_feature_names_out(), data=X_train_tfidf.toarray())

In [15]:
test_tfidf = pd.DataFrame(columns=tfidf.get_feature_names_out(), data=X_test_tfidf.toarray())

In [17]:
nb = BernoulliNB()

nb.fit(train_tfidf, y_train)

BernoulliNB()

In [47]:
soft_predict = nb.predict_proba(test_tfidf)#get probabilistic prediction
hard_predict = nb.predict(test_tfidf)#get hard prediction

In [31]:
y_test

139472    0
276148    0
590438    0
652460    1
333815    0
         ..
709691    1
620170    1
831781    1
539095    1
975882    1
Name: label, Length: 252679, dtype: int64

In [40]:
new_X = pd.DataFrame(columns=['comment'], data=X_test)

In [41]:
new_X.head()

,comment
139472,Trufant
276148,Too soon
590438,"Also, phone lights have no throw."
652460,"The Pirate Bay, of course!"
333815,Doesn't Sop have a Mac version though?


In [42]:
new_y = pd.DataFrame(columns=['label'], data=y_test)

In [43]:
new_y.head()

,label
139472,0
276148,0
590438,0
652460,1
333815,0


In [45]:
new_test = pd.concat([new_X, new_y], axis=1)
new_test.head()

,comment,label
139472,Trufant,0
276148,Too soon,0
590438,"Also, phone lights have no throw.",0
652460,"The Pirate Bay, of course!",1
333815,Doesn't Sop have a Mac version though?,0


In [49]:
test_predict = new_test.copy()

test_predict['soft_pred'] = np.round(soft_predict[:, 1], 2)
test_predict['hard_pred'] = hard_predict

In [53]:
total = test_predict.shape[0]

In [56]:
correct = test_predict.loc[test_predict['hard_pred'] == test_predict['label']].shape[0]

In [58]:
accuracy = correct / total *100
print(f'Correct prediction percentage: {round(accuracy, 2)}')

Correct prediction percentage: 65.56
